每次丟入新data時，注意有4個路徑要調整! path_background path_file npy_save png_save

In [4]:
# -*- coding: utf-8 -*-
"""
Created on Thu May  6 14:20:30 2021

@author: cinnamorolleo05g
"""
import os
import numpy as np
from pathlib import Path
import matplotlib.pyplot as plt
from scipy.fftpack import fft, ifft
from scipy.interpolate import CubicSpline
import time
'''
1. 一次讀一個frame.dat，所以記憶體佔用會小很多
2. 處理完後輸出成一張.png影像與一個.npy檔案，分別存在result_image與result_npy裡
兩種儲存方式的差別在於integer或float，integer這樣資料會小很多

'''
#路徑可動態調整
# path_background = r'\\\BOIL-NAS\homes\311514061\2024_6_13DC\BG'
path_background = r'D:\OCT\dental OCT\bare tooth\ensemble_model_aug\code\2024_8_13\BG'
 
# path_file = r'\\BOIL-NAS\homes\311514061\2024-5haveBG-SD-OCT dental calculus\2024dentalCalculus\supra8'

path_file = r'D:\OCT\dental OCT\bare tooth\ensemble_model_aug\code\2024_8_13\cal_raw\c8'

#畫圖處可有可無，故註解
# def plot_waveform(mean_BG):
#     plt.plot(mean_BG, linewidth=0.5)   # 一維所以直接 plot 他會自動抓x跟y
#     plt.show()

lines_per_frame = 2000 #共幾條A-line
total_length = 2048  # one A-line = 2048 pixels

number_BG = 10 #注意看有幾張BG，不夠的話要加到10

#根據folder的dat檔案數量調整:可以調超過frame的總數，不過跑完所有frame後找不到下一個會出現FileNotFoundError
number_OCT = 1000 #共幾個frame

# ============== BG subtraction ==============#True False
BG_subtraction = True

if BG_subtraction is True:
    data_10BG = np.zeros((lines_per_frame, total_length, number_BG))
    for num in range(1, number_BG +1):  #range(1, 11) 從 1 開始到 10
        file_background = os.path.join(path_background, 'frame%d.dat' % num) #
        temp = np.fromfile(file_background, dtype='>i2')  # > means_big endian, i2_means integer 16(16位元整數)
        data_10BG[:, :, num-1] = temp[4:].reshape(2000, 2048)   # 把多的前4筆資料移除後存入 data_10BG 
    mean_BG = np.mean(data_10BG, axis=2)
    A_line_mean_BG = np.mean(np.mean(data_10BG.reshape((2000, 2048, 10)), axis=2), axis=0).reshape(2048, 1)
    # plot_waveform(20*np.log10(np.abs(np.fft.fft(A_line_mean_BG[:, 0]))))

    matrix_mean_BG = np.tile(A_line_mean_BG, 2000).T    # 為了方便後面二維矩陣的處理，將 A_line_mean_BG 擴增為二維矩陣
    # plot_waveform(A_line_mean_BG) 

elif BG_subtraction is False:
    matrix_mean_BG = 0

''' 
load pixel calibration .txt，必須與現正執行的840_OCT_3_v2.ipynb檔放在同路徑
'''
# f=open(r'\\BOIL-NAS\homes\311514061\2024-5haveBG-SD-OCT dental calculus\2024dentalCalculus\p_avg_1.txt','r')

f=open(r'D:\OCT\dental OCT\bare tooth\ensemble_model_aug\code\dat_data_preprocessing\p_avg_1.txt','r')
p_avg_txt=f.readlines()
p_avg = np.zeros(len(p_avg_txt))  # 建立空矩陣儲存資料  
p_avg[:] = p_avg_txt[:]   # 把前面讀進來的data塞進 numpy矩陣裡 
base_line = np.arange(4096)
final_OCT = np.zeros((2000, 550, number_OCT))
test_fft = np.zeros((2000, 2048), dtype='complex_')
##############################################################################################################
''' 確認儲存結果的路徑是否存在，若不存在則建立
'''

#需調整
# npy_save = r'\\BOIL-NAS\homes\311514061\2024-5haveBG-SD-OCT dental calculus\2024dentalCalculus\supra8\npy'
# png_save = r'\\BOIL-NAS\homes\311514061\2024-5haveBG-SD-OCT dental calculus\2024dentalCalculus\supra8\png'

npy_save = r'D:\OCT\dental OCT\bare tooth\ensemble_model_aug\code\2024_8_13\cal_raw\c8\npy'
png_save = r'D:\OCT\dental OCT\bare tooth\ensemble_model_aug\code\2024_8_13\cal_raw\c8\png'

if Path(npy_save).exists() is False:
        os.makedirs(npy_save)         
if Path(png_save).exists() is False:
        os.makedirs(png_save)
##############################################################################################################

''' 開始一個frame一個frame讀入，並將處理完的結果儲存為圖片(整數矩陣)與npy(float矩陣)
'''
start = time.process_time()
for num in range(0, number_OCT):  # 從 0 開始到 frame的數量-1
    ''' 將會用到的矩陣變數先建立，同時用於每次完成迴圈後清空
    '''
    if num%50 == 0:
        print('Have processed: {}, total: {}.'.format(num, number_OCT))
    data_OCT = np.zeros((lines_per_frame, total_length))    # 一個frame輸出一次結果，因此只需要建立二維矩陣
    temp = np.zeros(2000*2048)
    OCT_interplation = np.zeros((2000, 4096), dtype='complex_')
    zero_padding = np.zeros((2000, 4096), dtype='complex_')
    inverse_ZeroPadding = np.zeros((2000, 2048), dtype='complex_') #why?
    fig_reshape = np.zeros((550, 2000))
    
    file_background = os.path.join(path_file, 'frame%d.dat' % (num +1)) #
    temp = np.fromfile(file_background, dtype='>i2')  # > means_big endian, i2_means integer 16(16位元整數)
    # data_OCT[:, :, num-1] = temp[4:].reshape(2000, 2048)
    data_OCT = temp[4:].reshape(2000, 2048) - matrix_mean_BG   # 把多的前4筆資料移除(標頭檔)並減去mean_BG後存入data_OCT
    # plot_waveform(data_OCT[0, :, num])
    
    temp_fft = fft(data_OCT)    
    
    # 將zero padding加入fft中間
    zero_padding[:, :1024] = temp_fft[:, :1024]
    zero_padding[:, 3072:] = temp_fft[:, 1024:]
    
    temp_ifft = ifft(zero_padding)
    # plot_waveform(temp_ifft[0, :])
    
    # 因為內插只能一維一維做，因此多一層for迴圈分別完成每一列的內差
    for num_interplation in range(lines_per_frame): 
        cubic_interplation = CubicSpline(p_avg, temp_ifft[num_interplation, :])
        OCT_interplation[num_interplation, :] = cubic_interplation(base_line).flatten()
        # if num_interplation == 0:
        #     plot_waveform(OCT_interplation[0, :])

   
    temp_interplation_fft = 20*np.log10 (np.abs(fft(OCT_interplation))) 
    #plot_waveform(temp_interplation_fft[0, :])
    
    # shift
    inverse_ZeroPadding[:, :1024] = temp_interplation_fft[:, 3072:]
    inverse_ZeroPadding[:, 1024:] = temp_interplation_fft[:, :1024]
    
    #plot_waveform(inverse_ZeroPadding[0, :])
    
    final_OCT = np.abs(inverse_ZeroPadding[:, 450:1000])    # 裁切最後的結果並且取絕對值變回 floating point 

    fig_reshape = np.rot90(final_OCT)
    # 按照 frame number進行儲存 -> 之後確認data後再調vmin vmax ，提升對比度
    plt.imsave(png_save + '/frame%d.png' %(num+1), fig_reshape, cmap = 'gray', vmin=50, vmax=80)
    np.save(npy_save + '/frame%d.npy' %(num+1),fig_reshape)
end = time.process_time()
print("執行時間：%f 秒" % (end - start))  

Have processed: 0, total: 1000.
Have processed: 50, total: 1000.
Have processed: 100, total: 1000.
Have processed: 150, total: 1000.
Have processed: 200, total: 1000.
Have processed: 250, total: 1000.
Have processed: 300, total: 1000.
Have processed: 350, total: 1000.
Have processed: 400, total: 1000.
Have processed: 450, total: 1000.
Have processed: 500, total: 1000.
Have processed: 550, total: 1000.
Have processed: 600, total: 1000.
Have processed: 650, total: 1000.
Have processed: 700, total: 1000.
Have processed: 750, total: 1000.
Have processed: 800, total: 1000.
Have processed: 850, total: 1000.
Have processed: 900, total: 1000.
Have processed: 950, total: 1000.


FileNotFoundError: [Errno 2] No such file or directory: 'D:\\OCT\\dental OCT\\bare tooth\\ensemble_model_aug\\code\\2024_8_13\\cal_raw\\c6\\frame1000.dat'